# EDA002をもとに予測精度を計算


In [485]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import folium

In [486]:
class CFG:
    make_folder = False
    note_num = "StackingOpt/EDA003"
    seed = 42


OUTPUT_DIR = f'H:/study/output/{CFG.note_num}/'

if CFG.make_folder:
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)


In [487]:
unique_id = ['10000095', '10000269', '1020000002', '1110000001', '1110000010', '1110000011', '1110000012', '1110000013', '1110000014', '1110000015', '1160000025', '1160000090', '1160000091', '1160000182', '1160000185', '1160000253', '1160000387', '1160000402', '1160000419', '1160000420', '1160000423', '1270000026', '1280000048', '1550000001', '1650000004', '1680000001', '1680000002', '1680000003', '1680000004', '1680000010', '1680000017', '1680000021', '1680000033', '1680000047', '1680000054', '1680000057', '1680000063', '1680000067', '1680000080', '1680000081', '1680000097', '1680000107', '1680000108', '1680000112', '1680000151', '1680000152', '1680000213', '1680000216', '1680000217', '1680000218', '1680000223', '1680000228', '1680000285', '1680000287', '1680000327', '1680000364', '2220000001', '2220000002', '2220000003', '2730000001', '2910000002', '3000000007', '3000000012', '3000000042', '5000000044', '5000000045', '6000000016', '6000000017', '6060000016', '6060000017', '6060000018', '6170000016', '6170000123', '6170000124', '6170000125', '6620000065', '6620000066', '6620000088', '6620000089', '6620000111', '6620000117', '6620000118', '6620000121', '6620000122', '6620000123', '6620000124', '6620000131', '6620000132', '6910000180', '6910000198', '6910000200', '6910000206', '6910000216', '6910000217', '6910000239', '6910000240', '6910000249', '6910000250', '6910000421', '6910000424', '6910000425', '6910000469', '6910000470']
unique_id = [int(i) for i in unique_id]
to_unique_id = [str(num).zfill(10) for num in unique_id]

# preprocessing_data\1_twoweeks_nv より

In [497]:
df_list = []
for id in to_unique_id:
    path = f"H:/study/preprocessing_data/1_twoweeks_nv/{id}.csv"
    df = pd.read_csv(path, header=None, index_col=None)
    df.columns = ["year","month","day","hour","flag","nv","two_weeks_max"]
    df["id"] = int(id)
    df_list.append(df)
    
df = pd.concat(df_list) 
df["year"] += 2000
df["minute"] = df["hour"]%1*60
df["hour"] = df["hour"]//1
df[["hour","minute"]] = df[["hour","minute"]].astype(int)
df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])

In [552]:
import utils

df = utils.get_preprocessing_data(to_unique_id)



# prediction_data\parameter_lambda_0.019_iterate_1000_p3\env

In [490]:
lat = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\lati_zenkoku.csv", header=None, index_col=None)
lon = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\long_zenkoku.csv", header=None, index_col=None)
id_all_data = pd.read_csv(r"H:\study\preprocessing_data\id_all_data.csv", encoding="shift_jis")
id_data = id_all_data[id_all_data.id.isin(unique_id)].reset_index(drop=True)


In [538]:
def get_prediction_data(id_data,lat,lon,dt):
    path = f"H:/study/prediction_data/parameter_lambda_0.019_iterate_1000_p3/env/{dt.month}月/{dt.month}月{dt.day}日/NV{dt.year%1000}{dt.month}{dt.day}{dt.hour}.{int(dt.minute/60*10)}_0.02_0.019_1000_p3_env.csv"
    df = pd.read_csv(path, header=None, index_col=None)

    min_lat,max_lat = id_data.id_lat.min(),id_data.id_lat.max()
    min_lng,max_lng = id_data.id_lng.min(),id_data.id_lng.max()
    userow = (lon.iloc[:,0]>=min_lng)&(lon.iloc[:,0]<=max_lng)
    usecol = (lat.iloc[0]>=min_lat)&(lat.iloc[0]<=max_lat)

    df = df.loc[userow,usecol]
    df = df.replace({ " nan": np.nan," -nan(ind)":np.nan})
    df = df.astype(float)

    pred_data = pd.DataFrame(zip(df.to_numpy().reshape(-1),\
                            lat.loc[userow,usecol].to_numpy().reshape(-1),\
                            lon.loc[userow,usecol].to_numpy().reshape(-1)),\
                            columns=["pred","id_lat_mesh","id_lng_mesh"])
    pred_data["datetime"] = dt
    return df,pred_data



In [539]:
import datetime

start_date = datetime.datetime(2013, 8, 15, 6, 30)
end_date = datetime.datetime(2014, 7, 31, 18, 0)
# start_date = datetime.datetime(2014, 7, 1, 6, 30)
# end_date = datetime.datetime(2014, 7, 31, 18, 0)

delta = datetime.timedelta(minutes=30)

date_range = []
date = start_date
while date <= end_date:
    if date.hour > 6 and date.hour < 18:
        date_range.append(date)
    elif date.hour == 18 and date.minute == 0:
        date_range.append(date)
    elif date.hour == 6 and date.minute == 30:
        date_range.append(date)
    date += delta


In [540]:
df_list,pred_data_list = [],[]
for i,dt in enumerate(date_range):
    df,pred_data = get_prediction_data(id_data,lat,lon,dt)
    df_list.append(df)
    pred_data_list.append(pred_data)
    if i%int(len(date_range)*0.1)==0:
        print(f"{i/len(date_range)*100}%_finish")

0.0%_finish
9.995251661918328%_finish
19.990503323836656%_finish
29.985754985754987%_finish
39.98100664767331%_finish
49.97625830959164%_finish
59.971509971509974%_finish
69.96676163342829%_finish
79.96201329534662%_finish
89.95726495726495%_finish
99.95251661918329%_finish


In [541]:
preds = pd.concat(pred_data_list).reset_index(drop=True)
mesh_data_list = df_list.copy()

# 予測精度を比較

In [542]:
df_list = []
for id in to_unique_id:
    path = f"H:/study/preprocessing_data/1_twoweeks_nv/{id}.csv"
    df = pd.read_csv(path, header=None, index_col=None)
    df.columns = ["year","month","day","hour","flag","nv","two_weeks_max"]
    df["id"] = int(id)
    df_list.append(df)
    
df = pd.concat(df_list) 

df["year"] += 2000
df["minute"] = df["hour"]%1*60
df["hour"] = df["hour"]//1
df[["hour","minute"]] = df[["hour","minute"]].astype(int)
df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])

In [543]:
df = df.merge(id_data[["id","id_lat_mesh","id_lng_mesh","pvrate","observed_max"]],on=["id"],how="left")
df = df[df.datetime.isin(date_range)]
df = df.merge(preds,on=["datetime","id_lat_mesh","id_lng_mesh"],how="left")

df["pred*two_weeks_max"] = df["pred"]*df["two_weeks_max"]
df["nv*twoweeks_max"] = df["nv"]*df["two_weeks_max"]

In [544]:
df["APE"] = np.abs(df["pred*two_weeks_max"]-df["nv*twoweeks_max"])/df["observed_max"]*100

In [545]:
df["APE"].mean()

4.622661015944398

In [547]:
df["month"] = df.datetime.dt.month
for month,group in df.groupby("month"):
    print(f"{month}月",group["APE"].mean() )
    print()

1月 3.004600552943324

2月 4.531547490686455

3月 3.4208157122279808

4月 4.661107656975637

5月 5.445719045449569

6月 6.36660266111151

7月 6.4246202475227125

8月 5.074576764676934

9月 5.984355188662475

10月 4.745663452336866

11月 3.1692568426963574

12月 2.7771447530777835



In [555]:
import utils

lat = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\lati_zenkoku.csv", header=None, index_col=None)
lon = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\long_zenkoku.csv", header=None, index_col=None)
id_all_data = pd.read_csv(r"H:\study\preprocessing_data\id_all_data.csv", encoding="shift_jis")

unique_id = ['10000095', '10000269', '1020000002', '1110000001', '1110000010', '1110000011', '1110000012', '1110000013', '1110000014', '1110000015', '1160000025', '1160000090', '1160000091', '1160000182', '1160000185', '1160000253', '1160000387', '1160000402', '1160000419', '1160000420', '1160000423', '1270000026', '1280000048', '1550000001', '1650000004', '1680000001', '1680000002', '1680000003', '1680000004', '1680000010', '1680000017', '1680000021', '1680000033', '1680000047', '1680000054', '1680000057', '1680000063', '1680000067', '1680000080', '1680000081', '1680000097', '1680000107', '1680000108', '1680000112', '1680000151', '1680000152', '1680000213', '1680000216', '1680000217', '1680000218', '1680000223', '1680000228', '1680000285', '1680000287', '1680000327', '1680000364', '2220000001', '2220000002', '2220000003', '2730000001', '2910000002', '3000000007', '3000000012', '3000000042', '5000000044', '5000000045', '6000000016', '6000000017', '6060000016', '6060000017', '6060000018', '6170000016', '6170000123', '6170000124', '6170000125', '6620000065', '6620000066', '6620000088', '6620000089', '6620000111', '6620000117', '6620000118', '6620000121', '6620000122', '6620000123', '6620000124', '6620000131', '6620000132', '6910000180', '6910000198', '6910000200', '6910000206', '6910000216', '6910000217', '6910000239', '6910000240', '6910000249', '6910000250', '6910000421', '6910000424', '6910000425', '6910000469', '6910000470']
unique_id = [int(i) for i in unique_id]
id_data = id_all_data[id_all_data.id.isin(unique_id)].reset_index(drop=True)

dt= datetime.datetime(2013, 8, 15, 6, 30)
folder_path = "H:/study/prediction_data/parameter_lambda_0.019_iterate_1000_p3/env/"
df,pred_data = utils.get_prediction_data(id_data,lat,lon,dt,folder_path)

TypeError: get_prediction_data() takes 4 positional arguments but 5 were given